In [1]:
#standard libraries
import pandas as pd, numpy as np

#randomization
import random

In [2]:
#export the file
enrolment = pd.read_excel('./experimental/tables_to_use/dashboard_sketches.xlsx',
                          sheet_name='enrolment')

enrolment.head(3)

,COURSE_ID,CYCLE_ID,STUDENT_ID,ENROLLMENT_DATE,COMPLETION_DATE,CANCELLED,CANCELLATION_REASON,Unnamed: 7,Unnamed: 8,Unnamed: 9,ADDENDUM,NEED
0,NaN,NaN,S000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COURSE_ID,X
1,NaN,NaN,S000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CYCLE_ID,X
2,NaN,NaN,S000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STUDENT_ID,X


In [3]:
#remove uninformative features
enrolment = enrolment.drop(['CYCLE_ID',
                            'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
                            'CANCELLATION_REASON', 'ADDENDUM', 'NEED'],
                           axis=1,
                           errors='ignore')

#drop features to be created at a later stage
enrolment = enrolment.drop(['ENROLLMENT_DATE', 'COMPLETION_DATE'],
                           axis=1,
                           errors='ignore')

In [4]:
enrolment.head(3)

,COURSE_ID,STUDENT_ID,CANCELLED
0,NaN,S000000,NaN
1,NaN,S000001,NaN
2,NaN,S000002,NaN


#### CREATE CANCELLED BY RANDOMIZATION

let's say a student cancels if their student_id is divisable by 7 without a remainder

In [5]:
#set the index
enrolment = enrolment.reset_index()

In [6]:
#rename the feature(s)
enrolment = enrolment.rename(columns={'index': 'ID',
                              })

In [7]:
#write a function to set a random number of students to cancelled if their id is divisable by 7 without a remainder
def cancelled_students(data):
    #if the ID is divisable by 7 without a remainder
    if data['ID'] % 7 == 0:
        return True
    else:
        return False

In [8]:
#apply the function
enrolment['CANCELLED'] = enrolment.apply(cancelled_students,
                            axis=1)

In [9]:
#check that the change was applied
enrolment['CANCELLED'].value_counts()

CANCELLED
False    171
True      29
Name: count, dtype: int64

that's 14.5% that have completed the studies for any reason

#### CREATE ENROLMENT_DATE BY RANDOMIZATION

In [10]:
#randomize year between 2023-24
enr_year = [random.randrange(2023, 2025, 1) for i in range(200)]

In [11]:
#randomize month between 1-12
enr_month = [random.randrange(1, 13, 1) for i in range(200)]

In [12]:
#randomize day between 1-30
enr_day = [random.randrange(1, 31, 1) for i in range(200)]

CHECK TO BE MADE FOR FEBRUARY (28 OR 29)

In [13]:
#insert new features
enrolment['ENROLMENT_DAY'] = None
enrolment['ENROLMENT_MONTH'] = None
enrolment['ENROLMENT_YEAR'] = None

enrolment = enrolment.assign(ENROLMENT_DAY=enr_day)
enrolment = enrolment.assign(ENROLMENT_MONTH=enr_month)
enrolment = enrolment.assign(ENROLMENT_YEAR=enr_year)

CHECK FOR FEBRUARY

In [14]:
#check February
enrolment.iloc[:, [4, 5, 6]][enrolment['ENROLMENT_MONTH'] == 2]

,ENROLMENT_DAY,ENROLMENT_MONTH,ENROLMENT_YEAR
11,8,2,2023
12,12,2,2024
14,10,2,2023
18,4,2,2023
21,14,2,2024
39,12,2,2023
42,8,2,2024
44,13,2,2024
49,28,2,2023
55,2,2,2023


In [15]:
# #make a change
# enrolment.iloc[117, 4] = 29

CREATE ENROLMENT DATE

In [16]:
#create a new feature
enrolment['ENROLMENT_DATE'] = pd.to_datetime((enrolment.ENROLMENT_YEAR*10000 + enrolment.ENROLMENT_MONTH*100
                                              + enrolment.ENROLMENT_DAY).apply(str),format="%Y%m%d")

#### CREATE COMPLETION_DATE BY RANDOMIZATION

In [17]:
#randomize day between 1-28
cmpl_day = [random.randrange(1, 29, 1) for i in range(200)]

In [18]:
#randomize month between 1-12
cmpl_month = [random.randrange(1, 13, 1) for i in range(200)]

In [19]:
#randomize year between 2023-24
cmpl_year = [random.randrange(2023, 2025, 1) for i in range(200)]

In [20]:
#insert new features
enrolment['COMPLETION_DAY'] = None
enrolment['COMPLETION_MONTH'] = None
enrolment['COMPLETION_YEAR'] = None

enrolment = enrolment.assign(COMPLETION_DAY=cmpl_day)
enrolment = enrolment.assign(COMPLETION_MONTH=cmpl_month)
enrolment = enrolment.assign(COMPLETION_YEAR=cmpl_year)

CREATE COMPLETION DATE

In [21]:
#create a new feature
enrolment['NEW_DATE'] = pd.to_datetime((enrolment.COMPLETION_YEAR*10000 + enrolment.COMPLETION_MONTH*100
                                        + enrolment.COMPLETION_DAY).apply(str),format="%Y%m%d")

In [22]:
#write a function for a simple condition
def create_condition(x):
    if x['ENROLMENT_DATE'] > x['NEW_DATE']:
        return True
    else:
        return False

enrolment['BOOL'] = enrolment.apply(create_condition,
                                    axis=1)

In [23]:
from datetime import datetime, date, timedelta

#write a function to create the completion date by comparing two dates
def create_date(x):
    if x['BOOL']:
        #add 25 additional days
        change = x['ENROLMENT_DATE'] + timedelta(days=25)
        return change
    else:
        return x['NEW_DATE']

enrolment['COMPLETION_DATE'] = enrolment.apply(create_date,
                                               axis=1)

LOGIC CHECK

enrolment date cannot be after completion date

In [24]:
enrolment[enrolment['ENROLMENT_DATE'] > enrolment['COMPLETION_DATE']]

,ID,COURSE_ID,STUDENT_ID,CANCELLED,ENROLMENT_DAY,ENROLMENT_MONTH,ENROLMENT_YEAR,ENROLMENT_DATE,COMPLETION_DAY,COMPLETION_MONTH,COMPLETION_YEAR,NEW_DATE,BOOL,COMPLETION_DATE


In [25]:
#drop uninformative features
enrolment = enrolment.drop(['ENROLMENT_DAY', 'ENROLMENT_MONTH', 'ENROLMENT_YEAR',
                            'COMPLETION_DAY','COMPLETION_MONTH', 'COMPLETION_YEAR',
                            'NEW_DATE', 'BOOL'],
                           axis=1,
                           errors='ignore')

In [26]:
enrolment.head(2)

,ID,COURSE_ID,STUDENT_ID,CANCELLED,ENROLMENT_DATE,COMPLETION_DATE
0,0,NaN,S000000,True,2023-12-22,2024-01-16
1,1,NaN,S000001,False,2023-08-27,2024-04-15


EXPORT THE TABLE

In [27]:
#create a file name
xlx_name = 'enrolment.xlsx'

#export to excel
enrolment.to_excel('./experimental/created_tables/' + xlx_name,
                   sheet_name='enrolment_details')